In [7]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import time, pickle, os

In [8]:
'''
# Defining rates & factors
epsilon = .9 # Epsilon greedy factor
minEpsilon = .1
maxEpsilon = 1
epsilonDecayRate = .0001

totalEpisodes = 10000
maxSteps = 100

lr = .81 # Learning rate
gamma = .96 # Discount factor
'''

'\n# Defining rates & factors\nepsilon = .9 # Epsilon greedy factor\nminEpsilon = .1\nmaxEpsilon = 1\nepsilonDecayRate = .0001\n\ntotalEpisodes = 10000\nmaxSteps = 100\n\nlr = .81 # Learning rate\ngamma = .96 # Discount factor\n'

In [9]:
def chooseAction(state, qTable, isTraining, env, epsilon):
    action = 0
    if isTraining and np.random.uniform(0,1) < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(qTable[state, :])
    return action

In [10]:
def updateQTable(state, action, newState, reward, qTable, lr, gamma):
    currentQ = qTable[state, action]
    newMaxQ = np.max(qTable[newState, :])
    #newMaxQ = np.argmax(qTable[newState, :])
    qTable[state, action] = currentQ + lr*(reward + gamma*newMaxQ - currentQ)
    return qTable

In [11]:
def runQLearning(totalEpisodes, isTraining = True, render = False):
    
    # Creating the environment
    env = gym.make("FrozenLake-v1", is_slippery = False, render_mode = 'human' if render else None)
    
    # Initialize Q-Table
    if isTraining:
        qTable = np.random.uniform(low = 0, high = 1, size = (env.observation_space.n, env.action_space.n)) # Q(s,a) initialized randomly
        #qTable = np.zeros((env.observation_space.n, env.action_space.n)) # Q(s,a) initialized with zeros
    else:
        # Load trained model Q-Table
        f = open('frozenLake_qTable_Q_Learning.pk1', 'rb')
        qTable = pickle.load(f)
        f.close()

    # Defining rates & factors
    epsilon = 1 # Epsilon greedy factor
    #minEpsilon = .1
    #maxEpsilon = 1
    epsilonDecayRate = .0001
    lr = .9 # Learning rate
    gamma = .9 # Discount factor

    rewards = np.zeros(totalEpisodes)

    # Start

    for episode in range(totalEpisodes):

        if isTraining:
            print("Episode: " + str(episode))
        else:
            print("Starting attempt...")
        
        terminated = False
        truncated = False

        state = env.reset()[0]
        
        while (not terminated and not truncated):

            action = chooseAction(state, qTable, isTraining, env, epsilon)
            
            newState, reward, terminated, truncated, _ = env.step(action)

            if isTraining:
                updateQTable(state, action, newState, reward, qTable, lr, gamma)

            state = newState

        #epsilon = minEpsilon + (maxEpsilon - minEpsilon) * np.exp(-epsilonDecayRate * episode)
        epsilon = max(epsilon - epsilonDecayRate, 0)

        # Stabilizing Q-Table when exploration has finished
        if(epsilon == 0):
            lr = .0001
     
        if reward == 1:
            rewards[episode] = 1
   
    env.close()

    if isTraining:
        print("Training completed. Environment closed")
    else:
        print("Attempt finished")

    # Rewards graphic
    sumRewards = np.zeros(totalEpisodes)
    for i in range(totalEpisodes):
        sumRewards[i] = np.sum(rewards[max(0, i-100):(i+1)]) #Rewards every 100 episodes
    if isTraining:
        plt.plot(sumRewards)
        plt.savefig("frozenLake_rewards.png")

    #print(qTable)

    # Saving the Q-Table after the training
    if isTraining:
        with open("frozenLake_qTable_Q_Learning.pk1", 'wb') as f:
            pickle.dump(qTable, f)

In [12]:

if __name__ == '__main__':
    runQLearning(15000)
    #runQLearning(1, isTraining=False, render=True)


Starting attempt...


c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Attempt finished
